In [4]:
from tkinter import *  # Ya incluye todos los elementos de tkinter
from tkinter import filedialog, messagebox  # Importación específica para messagebox y filedialog
from tkinter.ttk import Combobox  # Importar Combobox de ttk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
import numpy as np
import pandas as pd
import os
import tkinter



window = Tk()
window.title("Cosmic Ray Deleter")
window.geometry('850x650')    

#la ubicacion de la imagen de fondo 
imagen=PhotoImage(file='cosmicray.png')

#carpeta donde quieres que se abra el buscador y donde quieres que se abra cuando guardes
abrir=os.getcwd()
guardar=os.getcwd()


fondo=Label(window,image=imagen).place(x=50,y=100) 

ubicacion=StringVar()
data=StringVar()
intensidad=[]
wavelength=[]
flux_sin_spikes=[]

 
from tkinter import filedialog
from tkinter import scrolledtext
from tkinter.ttk import *


lbl = Label(window, text="Welcome to CRD", font=("Arial Bold", 20)).pack()#texto

lbl = Label(window, text="Enter the spectrum file path", font=("Arial Bold", 10)).pack()#texto

path2=[] #for save
fase=[]
ESPECTRO=[]
NEW_ESPECTRO=[]
cabezeras=[]

def leer_spec(path,ot=False,MJD=False,as_pandas=False,compress=False):

    if compress==True:
        import gzip

        with gzip.open(path, 'rb') as f2:
            espectro_lineas= [linea.split() for linea in f2]
    else:
        with open(path,'rt') as f2:
            espectro_lineas= [linea.split() for linea in f2]

    componente_filas_spec=[]
    spec=[]
    fases=[]
    iniciales=[]
    finales=[]
    alphas=[]
    for i in range(len(espectro_lineas)):
        if len(espectro_lineas[i])>1:
            if 'SPEC' in espectro_lineas[i][1] and 'NSPEC' not in espectro_lineas[i][1]:
                componente_filas_spec.append(i)  #me dice en que componente esta el spectro de la posicion

            if 'time:' == espectro_lineas[i][1] :
                fase=espectro_lineas[i][2]

                spec.append(float(fase)) #le saque el round porque me daba espectros repetidos cuando, si lo quiero poner deberia quiza usar un drop duplicates en el
                                            #calculo del rms y mad
                fases=spec
                #print(fase)
            else:
                if MJD==True:
                    if len(espectro_lineas[i])==5:
                        if 'MJD' in espectro_lineas[i][4] and 'SPEC' in espectro_lineas[i]:
                            fase=espectro_lineas[i][4]
                            fase=float(fase.strip('MJD='))
                            spec.append(fase) #saque el round
                            fases=spec
            if ot==True:
                if 'ini:' in espectro_lineas[i][1]:
                    inicial=espectro_lineas[i][2]
                    iniciales.append(float('%.3f'%float(inicial)))
                if 'fin:' in espectro_lineas[i][1]:
                    final=espectro_lineas[i][2]
                    finales.append(float('%.3f'%float(final)))
                if 'alpha:' in espectro_lineas[i][1]:
                    alpha=espectro_lineas[i][2]
                    alphas.append(float('%.3f'%float(alpha)))


    ESPECTRO=[]
    for j in range(len(componente_filas_spec)):
        #nombre_spec = str(spec[j])
        #print (nombre_spec)
        espectro1=[]
        if j==len(componente_filas_spec)-1:
            final=len(espectro_lineas)
            lineas_espectro=np.arange(componente_filas_spec[j]+1,final,1)
            #print (lineas_espectro)
            for i in lineas_espectro:
                if espectro_lineas[i][1]=='WAVE':
                    continue
                if float(espectro_lineas[i][1])!=0:
                    espectro1.append(espectro_lineas[i])

        else:
            if ot==True:
                res=4 #esto es para que lea los archivos bien ya que los OT tienen info extra
            else:
                res=1
            lineas_espectro=np.arange(componente_filas_spec[j]+1,componente_filas_spec[j+1]-res,1) #me dice las lineas que tengo q guardar por filtro
            #print (lineas_espectro)
            for i in lineas_espectro:

                if len(espectro_lineas[i])==0:
                    break
                if espectro_lineas[i][1]=='WAVE':
                    continue

                if float(espectro_lineas[i][1])!=0:
                    espectro1.append(espectro_lineas[i])
        if as_pandas==True:
            #print(len(espectro1[0]))
            #print(espectro1)
            try:
                if len(espectro1[0])==2:
                    espectro1=pd.DataFrame(espectro1,dtype="float64",columns=["wave","flux"])
                elif len(espectro1[0])==3:
                    espectro1=pd.DataFrame(espectro1,dtype="float64",columns=["wave","flux",'fluxerr'])
                else:
                    return print('Imposible llevar a pandas, revisa las columnas')
            except:
                espectro1=pd.DataFrame(espectro1,dtype="float64",columns=["wave","flux"])

        ESPECTRO.append(espectro1)
    #print('Phases:',fases)

    if ot==True:
        return ESPECTRO,fases,iniciales,finales,alphas
    else:
        return ESPECTRO,fases



def examinar():
    global path
    global ESPECTRO
    global NEW_ESPECTRO
    global fase


    # Abrir la ventana del explorador de archivos
    path = filedialog.askopenfilename(initialdir=abrir)

    # Verificar si el usuario seleccionó un archivo o no
    if not path:
        # Si el usuario cerró la ventana sin seleccionar archivo, no hacer nada
        #messagebox.showinfo("Candeled", "No se seleccionó ningún archivo. El espectro anterior permanece cargado.")
        return
    ESPECTRO=[]
    NEW_ESPECTRO=[]
    # Si se seleccionó un archivo, proceder con la carga
    ubicacion.set(path)

    directorio = str(path)
    with open(directorio, 'r') as f:
        espectro_lineas = [linea.split() for linea in f]

    DJ_SPEC = []
    componente_filas_spec = []
    fase = []

    ESPECTRO, fase = leer_spec(directorio, MJD=True)

    r.set(len(ESPECTRO))
    a = np.arange(0, len(ESPECTRO))
    a = tuple(a)
    combo['values'] = a
    combo.current(0)

    NEW_ESPECTRO = ESPECTRO

def Save_as():
    global wavelength
    global flux_sin_spikes
    global NEW_ESPECTRO


    path2=filedialog.asksaveasfilename(initialdir=guardar)
    directorio2=str(path2)

    outputFile = open(directorio2,'w')
    for i in range(len(NEW_ESPECTRO)):

        outputFile.write('\n')

        try:
            len(NEW_ESPECTRO[i])

            outputFile.write('#\tSPEC\t?\tat\tMJD='+str(fase[i]))
            outputFile.write('\n')
            outputFile.write('#\tWAVE\tFLUX\n')
            for j in range(len(NEW_ESPECTRO[i])):
                outputFile.write(str(NEW_ESPECTRO[i][j][0])+'\t'+str(NEW_ESPECTRO[i][j][1])+'\n')
        except TypeError:
            for ii in range(len(cabezeras[i])):
                outputFile.write(cabezeras[i][ii]+'\t')
            outputFile.write('#\tSPEC\t?\tat\tMJD='+str(fase[i]))
            outputFile.write('\n')
            outputFile.write('#\tWAVE\tFLUX\n')
            for j in range(len(ESPECTRO[i])):
                outputFile.write(str(ESPECTRO[i][j][0])+'\t'+str(ESPECTRO[i][j][1])+'\n')

#------------------------------------------------------------------    
Button(window, text="Browser", command=examinar).pack()

r=IntVar()

Label(window, text="\nQuantity of spectra").pack()
Entry(window, justify=CENTER, state='readonly', textvariable=r).pack()


lbl = Label(window, text="Select Spectra", font=("Arial Bold", 10)).pack()
combo = Combobox(window)
combo.pack()


#---------------------------------------------------------------------


eme=StringVar()
Label(window, text="Length Scale M: (2m+1)").pack()
Entry(window, justify="center", textvariable=eme).pack()
eme.set(3)

limite=StringVar()
Label(window, text="Limite").pack()
Entry(window, justify="center", textvariable=limite).pack()
limite.set(3.5)


chk_state = BooleanVar()
#chk_state.set(True) #set check state
chk = Checkbutton(window, text='z_scores?', var=chk_state).pack()

def on_key_press(event):
    print("you pressed {}".format(event.key))
    key_press_handler(event, canvas, toolbar)

def _quit():
    #window2.quit()     # stops mainloop
    window2.destroy()  # this is necessary on Windows to prevent
                    # Fatal Python Error: PyEval_RestoreThread: NULL tstate

def borrar():
    n2.set('')
def plot_spec():  
    global intensidad
    global wavelength
    global flux_sin_spikes #para hacerlo 0 cada vez q ploteo
    global fase

    wave=[]
    flux=[]
    try:
        j=int(combo.get())#int(n2.get())
    except ValueError:
        return messagebox.showinfo('Error', 'Select a Spectra')

    window2=Tk()
    window2.title('Plot Spec')

    #ESPECTRO[j]=sorted(ESPECTRO[j])
    for i in range(len(ESPECTRO[j])):
        wave.append(float(ESPECTRO[j][i][0]))
        flux.append(float(ESPECTRO[j][i][1]))
    flux=np.array(flux)
    wave=np.array(wave)


    #para generar el plot en otra ventana
    fig = plt.Figure(figsize=(12,6), dpi=100)
    ax1 = fig.add_subplot(111)
    ax1.plot(wave,flux)

    ax1.set_title('Spectra N°'+ str(j)+' Date = '+str(fase[j]))

    canvas = FigureCanvasTkAgg(fig, master=window2)  # A tk.DrawingArea.
    canvas.draw()
    canvas.get_tk_widget().pack(side=tkinter.TOP, fill=tkinter.BOTH, expand=1)

    toolbar = NavigationToolbar2Tk(canvas, window2)
    toolbar.update()
    canvas.get_tk_widget().pack(side=tkinter.TOP, fill=tkinter.BOTH, expand=1)

    ####
    intensidad=flux
    wavelength=wave

    flux_sin_spikes=[]
    new_espectro=[]
    for i in range(len(intensidad)):
        new_espectro.append([wavelength[i],intensidad[i]])

    j=int(combo.get())
    NEW_ESPECTRO[j]=new_espectro    




def modified_z_score(intensity):    
    median_int = np.median(intensity)
    mad_int = np.median([np.abs(intensity - median_int)])
    modified_z_scores = 0.6745 * (intensity - median_int) / mad_int
    return modified_z_scores



def fixer(): #auto
    global intensidad
    global wavelength
    global flux_sin_spikes
    global NEW_ESPECTRO

    # Validación: verificar que se haya ploteado un espectro (intensidad no esté vacía)
    if len(intensidad) == 0 or len(wavelength) == 0:
        return messagebox.showerror("Error", "No spectrum is loaded. Please plot a spectrum first.")

    # First we calculated ∇x(i):
    y = intensidad
    #------------------------------------------
    try:
        m = int(eme.get())
    except ValueError:
        return messagebox.showinfo('Error', 'Enter a length scale value')

    estado = chk_state.get()
    try:
        threshold = float(limite.get())
    except ValueError:
        return messagebox.showinfo('Error', 'Enter a Limit value')

    #-------------------------------------------
    n = 1
    intensity_modified_z_score = np.array(np.abs(modified_z_score(np.diff(y, n=n))))
    
    # Si se desea plotear los z_scores
    if estado:
        window6 = Toplevel(window)
        window6.title('Z Scores')

        fig = plt.Figure(figsize=(12, 6), dpi=100)
        ax1 = fig.add_subplot(111)

        ax1.plot(wavelength[n:], intensity_modified_z_score)
        ax1.plot(wavelength[n:], threshold * np.ones(len(wavelength[n:])), label='Threshold')
        ax1.set_xlabel('Wavelength (nm)', fontsize=20)
        ax1.set_ylabel('Score', fontsize=20)

        ax1.legend()
        ax1.set_title('Modified Z-Score of ∇x(i)', fontsize=20)

        canvas = FigureCanvasTkAgg(fig, master=window6)  # A tk.DrawingArea.
        canvas.draw()
        canvas.get_tk_widget().pack(side=tkinter.TOP, fill=tkinter.BOTH, expand=1)

        toolbar = NavigationToolbar2Tk(canvas, window6)
        toolbar.update()
        canvas.get_tk_widget().pack(side=tkinter.TOP, fill=tkinter.BOTH, expand=1)

    #--------
    spikes = abs(np.array(modified_z_score(np.diff(y, n=n)))) > threshold  # Identificamos spikes

    y_out = y.copy()  # Copiamos los valores para no sobrescribir los originales
    prueba_x = []
    prueba_y = []

    for i in np.arange(len(spikes)):
        if spikes[i] != 0:  # Si hay un spike en la posición i
            if wavelength[i] < max(wavelength) - 20:
                
                # Aquí ajustamos los límites de w para que no se salga de rango
                w = np.arange(max(0, i - m), min(i + 1 + m, len(spikes)))  # Seleccionamos 2 m + 1 puntos alrededor del spike

                # Nos quedamos con los que no son spikes en la vecindad
                w2 = w[spikes[w] == 0]

                # Si no encontramos puntos que no sean spikes, incrementamos m hasta encontrar alguno
                while len(w2) == 0 and m < len(spikes):
                    m += 1
                    w = np.arange(max(0, i - m), min(i + 1 + m, len(spikes)))
                    w2 = w[spikes[w] == 0]

                if len(w2) > 0:  # Aseguramos que w2 no esté vacío
                    y_out[i] = np.abs(np.mean(y[w2]))  # Promediamos los valores

                    # Guardamos los valores no spikes para visualización (opcional)
                    for j in range(len(w2)):
                        prueba_x.append(wavelength[w2[j]])
                        prueba_y.append(y[w2[j]])

    flux_sin_spikes = y_out  # Actualizamos la salida sin spikes

    new_espectro = []
    for i in range(len(flux_sin_spikes)):
        new_espectro.append([wavelength[i], flux_sin_spikes[i]])

    j = int(combo.get())
    NEW_ESPECTRO[j] = new_espectro
    a = pd.DataFrame(NEW_ESPECTRO[j])

    # plot---------------------------------------------------------------------------------------
    window3 = Toplevel(window)
    window3.title('Automatic Fix')

    fig = plt.Figure(figsize=(12, 6), dpi=100)
    ax1 = fig.add_subplot(111)
    ax1.plot(wavelength, intensidad, label='Original')
    ax1.plot(wavelength, flux_sin_spikes, label='Despiked')

    ax1.legend()
    ax1.set_title('Despiked Spectrum')

    canvas = FigureCanvasTkAgg(fig, master=window3)  # A tk.DrawingArea.
    canvas.draw()
    canvas.get_tk_widget().pack(side=tkinter.TOP, fill=tkinter.BOTH, expand=1)

    toolbar = NavigationToolbar2Tk(canvas, window3)
    toolbar.update()
    canvas.get_tk_widget().pack(side=tkinter.TOP, fill=tkinter.BOTH, expand=1)
    #--------------------------------------------------------------------------------------------

start=IntVar()
end=IntVar()

def Fix(): #manual
    global intensidad
    global wavelength
    global flux_sin_spikes

    # Validación: verificar que end sea mayor que start
    if start.get() >= end.get():
        return messagebox.showerror("Error", "'End' must be greater than 'Start'")

    # Validación: verificar que se haya ploteado un espectro (intensidad no esté vacía)
    if len(flux_sin_spikes)==0:
        return messagebox.showerror("Error", "No spectrum is loaded. Please plot a spectrum first.")

    wave_palplot=wavelength

    if len(flux_sin_spikes)!=0: #esto es para comenzar con el sin spikes del automatico (si hay)
        flux_manual=flux_sin_spikes.copy()
    else:
        flux_manual=intensidad.copy()

    a=float(start.get())
    b=float(end.get())

    for i in range(len(wavelength)):
        if a>=wavelength[i]:
            flux1=intensidad[i]
            wave1=wavelength[i]
        if b>=wavelength[i]:
            flux2=intensidad[i]
            wave2=wavelength[i]

    pol=np.polyfit(np.array([wave1,wave2]),np.array([flux1,flux2]),deg=1)
    eva=np.arange(wave1,b+1,1)
    values=np.polyval(pol, eva)
    contador=0
    for i in range(len(wavelength)):
        if wavelength[i]>=a and wavelength[i]<=b:
            try:
                flux_manual[i]=values[contador]
                wavelength[i]=eva[contador]
                contador=contador+1
            except IndexError:
                continue

    flux_sin_spikes=flux_manual   
    new_espectro=[]
    for i in range(len(flux_sin_spikes)):
        new_espectro.append([wavelength[i],flux_sin_spikes[i]])

    j=int(combo.get())
    NEW_ESPECTRO[j]=new_espectro    

    #plot---------------------------------------------------------------------------------------
    window4=Tk()
    window4.title('Manual Fix')

    fig = plt.Figure(figsize=(12,6), dpi=100)
    ax1 = fig.add_subplot(111)
    #ax1.plot(eva,values,'-',label='eva')
    ax1.plot(wave_palplot,intensidad,'-.',label='Original')
    ax1.plot(wavelength,flux_manual,'-.',label='Despiked')
    #ax1.axvline(x=a,ls='--',color='red')
    #ax1.axvline(x=b,ls='--',color='red')

    ax1.legend()

    ax1.set_title('Despiked Spectrum')

    canvas = FigureCanvasTkAgg(fig, master=window4)  # A tk.DrawingArea.
    canvas.draw()
    canvas.get_tk_widget().pack(side=tkinter.TOP, fill=tkinter.BOTH, expand=1)

    toolbar = NavigationToolbar2Tk(canvas, window4)
    toolbar.update()
    canvas.get_tk_widget().pack(side=tkinter.TOP, fill=tkinter.BOTH, expand=1)

    #canvas.mpl_connect("key_press_event", on_key_press)



def Help():
    # Create help window
    window5 = Toplevel(window)  # Using Toplevel to open a secondary window without restarting the main app
    window5.title("Instructions")
    window5.geometry('800x500')
    
    # Title
    lbl = Label(window5, text="HELP", font=("Arial", 15))
    lbl.pack(pady=10)

    # Instructions
    help_text = """
    Welcome to Cosmic Ray Deleter:
    
    1. Press 'Browser' to select a spectrum file.
    2. Use the dropdown menu to select the spectrum to visualize.
    3. Press 'Plot Spec' to plot the selected spectrum.
    4. If you want to automatically remove cosmic rays, use 'Delete Cosmic Rays (Auto)'.
    5. To save the corrected spectrum, press 'Save as'.
    6. If the automatic correction does not work, you can use manual correction.
    7. For manual correction, enter the wavelength range where the line is located.
    8. Manual correction applies linear interpolation between both ends of the range.
    
    Parameters:
    - Length Scale M: Defines the length scale for filtering.
    - Limit: Adjusts the threshold for removing cosmic rays.
    - Z_Scores: Enables the option to visualize z-scores in the plot.
    
    If you need further assistance, contact the developer.
    m.ramirezvalenzuela@gmail.com
    """
    
    # Label with the instructions
    lbl = Label(window5, text=help_text, font=("Arial", 10), justify=LEFT)
    lbl.pack(padx=10, pady=10)

    # Close button for the help window
    Button(window5, text="Close", command=window5.destroy).pack(pady=10)



#------------------------------------------------------------------------------------------------

Button(window, text="Plot Spec", command=plot_spec).pack()

Button(window, text="Delete Cosmics rays (Auto)", command=fixer).pack()


lbl = Label(window, text="", font=("Arial Bold", 15)).pack()
lbl = Label(window, text="For Manual Fix",font=("Arial Bold", 15)).pack()


Label(window, text="Start").pack()
Entry(window, justify="center", textvariable=start).pack()

Label(window, text="End").pack()
Entry(window, justify="center",textvariable=end).pack()

Button(window, text="Manual Fix", command=Fix).pack()

Button(window, text="Save as", command=Save_as).pack()


Label(window, text="\nPATH").pack()
Entry(window, justify=CENTER, state='readonly',width=100, textvariable=ubicacion).pack()

lbl = Label(window, text="@Mauricio Ramirez v1.1 2021", font=("Arial", 15)).pack(side=BOTTOM)
Button(window,text='Help',command=Help).pack(side=BOTTOM)

def run(self):
    self.root.protocol("WM_DELETE_WINDOW", lambda: self.quit())
def quit(self):
    self.root.destroy()



window.mainloop()
#window.wm_protocol("WM_DELETE_WINDOW", lambda: self.quit())